In [ ]:
import pandas as pd
import warnings
from modules.FeatureExtractor import FeatureExtractor

warnings.filterwarnings("ignore")

In [21]:
df = pd.read_csv('processed/442100110323_standard.csv')
df

,READ_DATE,KWH_EXP
0,2019-01-01 00:00:00,2.334143
1,2019-01-01 00:15:00,2.344050
2,2019-01-01 00:30:00,2.500311
3,2019-01-01 00:45:00,2.432096
4,2019-01-01 01:00:00,2.388465
...,...,...
189883,2024-05-31 22:45:00,69.132420
189884,2024-05-31 23:00:00,69.132420
189885,2024-05-31 23:15:00,68.774401
189886,2024-05-31 23:30:00,68.765215


In [22]:
df_weather = pd.read_csv('processed/weather_2019_2024_standard.csv')

df_weather.interpolate(method='linear', inplace=True)

In [ ]:
# gabung data weather dengan data daily
# data weather merupakan data harian, sedangkan data daily merupakan data 15 menitan
# sehingga data weather akan digabung ke data harian dengan cara mengflatkan data 15 menitan dalam 1 hari dengan data weather

df['READ_DATE'] = pd.to_datetime(df['READ_DATE'])
df_weather['Tanggal'] = pd.to_datetime(df_weather['Tanggal'])

df['READ_DATE_day'] = df['READ_DATE'].dt.date
df_weather['Tanggal'] = df_weather['Tanggal'].dt.date

df = pd.merge(df, df_weather, how='left', left_on='READ_DATE_day', right_on='Tanggal')
df.drop(columns=['Tanggal'], inplace=True)

In [24]:
df.set_index('READ_DATE', inplace=True)
df.drop(columns=['READ_DATE_day', 'Unnamed: 0'], inplace=True)

In [25]:
df.isna().sum()

KWH_EXP    0
Tavg       0
RH_avg     0
RR         0
ss         0
dtype: int64

In [26]:
df.to_csv('processed/442100110323_standard_weather.csv')

In [ ]:
fe = FeatureExtractor(df, 'KWH_EXP')
df_features = (
    fe.create_lag_features()
      .create_rolling_features()
      .create_expanding_features()
      .create_date_features()
      .create_cyclical_features()
      .create_decomposition_features()
      .fill_missing_values()
      .get_features()
)

In [35]:
df_features

,KWH_EXP,Tavg,RH_avg,RR,ss,lag_1,lag_3,lag_7,lag_28,rolling_mean_1,...,season_cos,dayofweek_sin,dayofweek_cos,weekofyear_sin,weekofyear_cos,dayofyear_sin,dayofyear_cos,trend,seasonal,residual
READ_DATE,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2.334143,25.7,96.0,12.9,7.5,0.000000,0.000000,0.000000,0.000000,2.334143,...,1.0,0.866025,0.5,0.120537,0.992709,0.017166,0.999853,7.684237,-0.017837,-0.000047
2019-01-01 00:15:00,2.344050,25.7,96.0,12.9,7.5,2.334143,0.000000,0.000000,0.000000,2.344050,...,1.0,0.866025,0.5,0.120537,0.992709,0.017166,0.999853,7.684237,-0.011514,-0.000047
2019-01-01 00:30:00,2.500311,25.7,96.0,12.9,7.5,2.344050,0.000000,0.000000,0.000000,2.500311,...,1.0,0.866025,0.5,0.120537,0.992709,0.017166,0.999853,7.684237,0.008583,-0.000047
2019-01-01 00:45:00,2.432096,25.7,96.0,12.9,7.5,2.500311,2.334143,0.000000,0.000000,2.432096,...,1.0,0.866025,0.5,0.120537,0.992709,0.017166,0.999853,7.684237,0.064360,-0.000047
2019-01-01 01:00:00,2.388465,25.7,96.0,12.9,7.5,2.432096,2.344050,0.000000,0.000000,2.388465,...,1.0,0.866025,0.5,0.120537,0.992709,0.017166,0.999853,7.684237,0.045630,-0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 22:45:00,69.132420,29.8,68.0,0.0,10.8,69.132420,69.132420,65.469139,63.243948,69.132420,...,-1.0,-0.866025,-0.5,0.464723,-0.885456,0.507415,-0.861702,7.684237,-0.035359,-0.000047
2024-05-31 23:00:00,69.132420,29.8,68.0,0.0,10.8,69.132420,69.132420,65.559856,63.452988,69.132420,...,-1.0,-0.866025,-0.5,0.464723,-0.885456,0.507415,-0.861702,7.684237,-0.003232,-0.000047
2024-05-31 23:15:00,68.774401,29.8,68.0,0.0,10.8,69.132420,69.132420,66.302510,63.512601,68.774401,...,-1.0,-0.866025,-0.5,0.464723,-0.885456,0.507415,-0.861702,7.684237,0.030413,-0.000047


In [36]:
df_features.to_csv('processed/442100110323_standard_weather_features.csv')